# Key Performance Indicator (KPI) Calculation Pipeline 

# To do: clean the notebook so it is easy to read

### Goal of the notebook

This notebook takes in 
(1).Aimsun network(nodes and sections) 
(2).scraped Point of Interests in Fremont area 
(3).Aimsun Microsimulation outputs 
to calculate KPI of interest (eg. accessibility) and generate easily understandable heatmaps and isochrones in Kepler.gl. 

### Inputs of the pipeline:

- Raw Aimsun network data (only load notes and sections): in the dropbox folder under /Data processing/Raw/Network/Aimsun/
- Scraped Point of Interests (POIs) in Fremont area (a csv file): in the dropbox folder under /Data processing/Raw/Network/KPIs
- The Aimsun Microsimulation outputs have already been transfered from SGLite to csv, and stored in dropbox folder: /Aimsun/Outputs/vehSectTrajectory.csv

### Dependent libraries:

- os
- sys
- fremontdropbox
- geopandas
- shapely.geometry (Points and Linsting manipulation)
- keplergl (visualization)
- pandas
- networkx (network manipulation)
- pandana (accessibility measurement)
- warnings (supprese unnecessaryly warnings)

### Deliverables

The accessibility heatmaps and isochrones are generated along the way through Kepler.gl.

## Table of Contents
1. [Loading Aimsun network](#aim_net)
2. [Loading and Processing the scraped POIs](#POIs)
3. [Using Pandana package to plot the accessibility heatmap](#Pandana)
4. [Loading the micro-simulation results to Pandana network](#sim_results)
5. [Assign each POI to its nearest Aimsun node](#matching)
6. [Measuring accessibility and generate heatmap](#accessibility)
7. [Calculating accessility and generate isochrone](#isochrone)

In [1]:
# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account



dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

aux_files = data_path+'/Data processing/Auxiliary files'

# Processing output path
output_path = aux_files + '/OD demand'

In [2]:
# Read more about GeoPandas data structures here: http://geopandas.org/data_structures.html
import geopandas as gpd # To work with spatial data in a DataFrame
from shapely.geometry import Point, LineString # To create line geometries that can be used in a GeoDataFrame
from keplergl import KeplerGl
import pandas as pd

# packages for network manipulation and visualization
#import csv
#from operator import itemgetter
import networkx as nx
#import osmnx as ox
#import matplotlib.pyplot as plt
#import matplotlib.lines as mlines
#import cartopy.crs as ccrs
#from IPython.display import Image
#from mpl_toolkits.basemap import Basemap as Basemap #Installation failed
#from networkx.algorithms import community #This part of networkx, for community detection, needs to be imported separately.


#pandana
# import pandana as pdna
# from pandana.loaders import osm
%matplotlib inline
#from descartes import PolygonPatch
#import random
# import warnings
# warnings.filterwarnings('ignore')

#change the projection of the code
#import pyproj as proj

## Loading Aimsun network
<a id="aim_net"> </a>

- Geopandas (to read and convert the files) 
- Networkx (to convert the data to a network) 
- Visualize the Geopandas network shapefile in Kepler.gl

In [3]:
def read_gdf(path):
    gdf = gpd.GeoDataFrame.from_file(path)
    gdf = gdf.to_crs(epsg=4326)
    return gdf

Aimsun_nodes = read_gdf(data_path + "/Data processing/Raw/Network/Aimsun/nodes.shp")
Aimsun_sections = read_gdf(data_path + "/Data processing/Raw/Network/Aimsun/sections.shp")

In [4]:
print("Number of sections: " + str(Aimsun_sections.id.count()))
print("Number of nodes: " + str(Aimsun_nodes.id.count()))

Number of sections: 5626
Number of nodes: 2013


In [5]:
print("Sections")
print(Aimsun_sections.head(3))
print()
print("Nodes")
print(Aimsun_nodes.head(3))

Sections
      id  eid  name  nb_lanes  speed  capacity  rd_type  func_class   fnode  \
0  242.0  242  None         1  120.0    2100.0    175.0           1  9845.0   
1  243.0  243  None         3  104.0    6300.0    175.0           1  9852.0   
2  244.0  244  None         3  104.0    6300.0    175.0           1  9850.0   

    tnode                                           geometry  
0  9923.0  LINESTRING (-121.92244 37.49593, -121.92242 37...  
1  9848.0  LINESTRING (-121.92313 37.49526, -121.92173 37...  
2  9852.0  LINESTRING (-121.92352 37.49561, -121.92313 37...  

Nodes
       id   eid  name  nodetype                     geometry
0  9845.0  9845  None       3.0  POINT (-121.92249 37.49593)
1  9848.0  9848  None       0.0  POINT (-121.92173 37.49401)
2  9850.0  9850  None       0.0  POINT (-121.92353 37.49561)


## To do: fill NaN values with new nodes

In [6]:
# To do here
print(Aimsun_nodes.eid.unique())

['9845' '9848' '9850' ... '62848' '62852' None]


In [7]:
# Keeping only some columns
edges_topo = Aimsun_sections[['id', 'fnode', 'tnode', 'capacity', 'speed']]

edges_topo = edges_topo[edges_topo['fnode'].notna()]
edges_topo = edges_topo[edges_topo['tnode'].notna()]

edges_topo = edges_topo.rename(columns={"fnode": "id_node_source", "tnode": "id_node_target"})
edges_topo = edges_topo.astype(int)

print(edges_topo.head(3))

    id  id_node_source  id_node_target  capacity  speed
0  242            9845            9923      2100    120
1  243            9852            9848      6300    104
2  244            9850            9852      6300    104


In [8]:
print("New number of sections: " + str(edges_topo.id.count()))

New number of sections: 4318


In [9]:
# Uncomment to render the data

# map_1 = KeplerGl(height=1000)
# map_1.add_data(data=Aimsun_sections, name = "sections")
# map_1.add_data(data=Aimsun_nodes, name = "nodes")
# map_1

## Loading and Processing the Point of Interests
<a id="POIs"> </a>

In [10]:
#Point of Interests scraped from Google API
POIs = pd.read_csv(data_path + "/Data processing/Raw/Network/KPIs/locations_crawl.csv")
# Creating a Geographic data frame for Point of Interests
POIs_gdf = gpd.GeoDataFrame(POIs, crs=crs_degree, geometry=gpd.points_from_xy(POIs.Long, POIs.Lat))

POIs_gdf.head(3)

,Unnamed: 0,Names,Types,Lat,Long,geometry
0,0,Fremont,"['locality', 'political']",37.548540,-121.988583,POINT (-121.98858 37.54854)
1,1,ProCreativeWriters,"['point_of_interest', 'establishment']",37.504377,-121.964423,POINT (-121.96442 37.50438)
2,2,Baylands,"['neighborhood', 'political']",37.485034,-121.964375,POINT (-121.96437 37.48503)


In [12]:
nodes = Aimsun_nodes[['id']].astype(int)
nodes['x'] = Aimsun_nodes.geometry.x
nodes['y'] = Aimsun_nodes.geometry.y
nodes = nodes.rename(columns={"id": "id_node"})

In [13]:
print("Sections")
print(edges_topo.head(3))
print()
print("Nodes")
print(nodes.head(3))

Sections
    id  id_node_source  id_node_target  capacity  speed
0  242            9845            9923      2100    120
1  243            9852            9848      6300    104
2  244            9850            9852      6300    104

Nodes
   id_node           x          y
0     9845 -121.922491  37.495935
1     9848 -121.921727  37.494013
2     9850 -121.923525  37.495613


In [50]:
print("Number of sections: " + str(edges_topo.id.count()))
print("Number of nodes: " + str(nodes.id_node.count()))

print("Number of end nodes: " + str(edges_topo.id_node_target.unique().shape[0]))
print("Number of start nodes: " + str(edges_topo.id_node_source.unique().shape[0]))

Number of sections: 4318
Number of nodes: 2013
Number of end nodes: 2000
Number of start nodes: 2000


## Loading simulation results

### To do: 
1. Check MISECT DATABASE instead of MISECTTrajectory

In [51]:
vehSectTraj = pd.read_csv(data_path + '/Aimsun/Outputs/vehSectTrajectory.csv')

In [52]:
print("Number of datapoints in the simulation: : " + str(len(vehSectTraj)))
print("Number of sections in the simulation: : " + str(vehSectTraj.sectionId.unique().shape[0]))

vehSectTraj.head()

Number of datapoints in the simulation: : 1539569
Number of sections in the simulation: : 3073


,Unnamed: 0,did,oid,ent,sectionId,exitTime,travelTime,delayTime
0,0,27989,1,1,2873,50420.3,19.33450,0.000000
1,1,27989,1,2,2874,50433.3,12.74100,0.065405
2,2,27989,1,3,2880,50435.1,1.84495,0.000000
3,3,27989,1,4,7155,50444.2,9.11629,0.000000
4,4,27989,1,5,9212,50447.5,3.23939,0.000000


In [53]:
max_tt = vehSectTraj.travelTime.max()
mean_tt = vehSectTraj.travelTime.mean()

print(max_tt)
print(mean_tt)

6730.82
21.536573605930496


In [54]:
# Would be changed depending on the specific scenario: currently, I am using the travelTime of each section by the first car
vehSectTraj_temp = vehSectTraj.groupby("sectionId").mean() # was first before

vehSectTraj_temp.head()

,Unnamed: 0,did,oid,ent,exitTime,travelTime,delayTime
sectionId,,,,,,,
242,5.323860e+05,27989.0,19612.531982,24.076351,59995.530405,26.016289,19.982331
243,6.488195e+05,27989.0,24440.117725,17.790915,61953.610201,6.407848,0.116653
244,6.492795e+05,27989.0,24459.738245,16.789826,61953.859779,1.824515,0.079628
248,9.508395e+05,27989.0,37096.313725,3.000000,64670.654902,11.845910,8.478374
249,1.174674e+06,27989.0,44143.833333,29.333333,67056.750000,5.157480,2.083440


In [64]:
edge_weighted = pd.merge(edges_topo,
                           vehSectTraj_temp,
                           left_on='id',
                           right_on='sectionId',
                           how='left',
                           sort=True)

edge_weighted.travelTime.fillna(mean_tt, inplace = True)

# print(edge_weighted[edge_weighted.travelTime.isna()].head())
print(edge_weighted[edge_weighted.exitTime.isna()].head(2))
print(edge_weighted.head(2))

     id  id_node_source  id_node_target  capacity  speed  Unnamed: 0  did  \
18  268            9957           14770       700     50         NaN  NaN   
19  270           14770            9957       700     50         NaN  NaN   

    oid  ent  exitTime  travelTime  delayTime  
18  NaN  NaN       NaN   21.536574        NaN  
19  NaN  NaN       NaN   21.536574        NaN  
    id  id_node_source  id_node_target  capacity  speed     Unnamed: 0  \
0  242            9845            9923      2100    120  532386.041892   
1  243            9852            9848      6300    104  648819.540560   

       did           oid        ent      exitTime  travelTime  delayTime  
0  27989.0  19612.531982  24.076351  59995.530405   26.016289  19.982331  
1  27989.0  24440.117725  17.790915  61953.610201    6.407848   0.116653  


In [66]:
## Side work: plot sections that are not used

# Aimsun_sections_simulation = pd.merge(Aimsun_sections,
#                            vehSectTraj_temp,
#                            left_on='id',
#                            right_on='sectionId',
#                            how='left',
#                            sort=True)

# map_1 = KeplerGl(height=1000)
# map_1.add_data(data=Aimsun_sections_simulation[Aimsun_sections_simulation.travelTime.isnull()], name = "Sections without traffic")
# map_1.add_data(data=Aimsun_sections_simulation[Aimsun_sections_simulation.travelTime.notna()], name = "Sections with traffic")
# map_1

## To do: fix the code such that the two numbers below are equal

In [69]:
print("Number of Aimsun sections: " + str(Aimsun_sections.id.count()))
print("Number of sections used (after dropna in Aimsun section): " + str(edge_weighted.id.count()))
print("Number of sections with travel time after adding simulation values: " + str(edge_weighted.delayTime.count()))
print("Number of sections with travel time after adding mean to NaN values: " + str(edge_weighted.travelTime.count()))


Number of Aimsun sections: 5626
Number of sections used (after dropna in Aimsun section): 4318
Number of sections with travel time after adding simulation values: 2391
Number of sections with travel time after adding mean to NaN values: 4318


## Loading the edge weights into the Pandana network sections
## To do: put this later because Theo does not have pandana, and would like to be able to run everything without having errors (but it after BFS with Networkx
<a id="sim_results"> </a>

In [ ]:
# This is not correct
edge_weighted = edge_weighted[edge_weighted.travelTime.notna()]

In [ ]:
print(nodes_gdf["x"].count())

In [ ]:
#initialize the Pandana network 
net = pdna.Network(node_x = nodes_gdf["x"],
                   node_y = nodes_gdf["y"],
                   edge_from = edge_weighted["id_node_source"], 
                   edge_to = edge_weighted["id_node_target"],
                   edge_weights = edge_weighted[["travelTime"]])

In [ ]:
#scraped Point of Interests in Fremont Area
POIs_gdf.head()

In [ ]:
#only keep the POIs within Aimsun network
bounding_box = Aimsun_sections.unary_union.envelope
tem_df = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
intersections = gpd.overlay(POIs_gdf, tem_df, how='intersection')

In [ ]:
#save the scraped Point of Interests to shapefile
intersections.to_file(driver = 'ESRI Shapefile', filename= data_path + "/Data processing/Raw/Network/Aimsun/POI.shp")

In [ ]:
#visualize the POIs 
map_2 = KeplerGl(height=1000)
map_2.add_data(data=intersections, name = "Point Of Interests")
map_2.add_data(data=Aimsun_sections, name = "sections")
map_2.add_data(data=Aimsun_nodes, name = "nodes")
map_2




## Assign each POI to corresponding nearest Aimsun node
<a id="matching"> </a>

## To do: Maybe KDTree can be done without loading the network in Pandana

In [ ]:
#get_node_ids uses the KDTree from scipy --> Make it explicit that this is the important line here
near_ids = net.get_node_ids(intersections['Long'],
                            intersections['Lat'])

# Set the response as a new column on the POI reference df
intersections['nearest_node_id'] = near_ids

In [ ]:
nodes_gdf.index = range(len(nodes_gdf))

In [ ]:
# Create a merged dataframe that holds the node data (esp. x and y values)
# that relate to each nearest neighbor of each POI
nearest_to_pois = pd.merge(intersections,
                           nodes_gdf,
                           left_on='nearest_node_id',
                           right_on='id_node',
                           how='left',
                           sort=False,
                           suffixes=['_from', '_to'])

In [ ]:
#showing how each POI is matched to the nearest node id
nearest_to_pois.head(5)

In [ ]:
POI_matching_lines = []
for row_id, row in nearest_to_pois.iterrows():
    linestr = LineString([(row['geometry'].x, row['geometry'].y),
                          (row['x'], row['y'])]).buffer(0.000001)
    POI_matching_lines.append(linestr)
    

In [ ]:
POI_matching_lines_gdf = pd.DataFrame({'geometry':POI_matching_lines})
POI_matching_lines_gdf = gpd.GeoDataFrame(POI_matching_lines_gdf, crs=crs)

In [ ]:
POI_matching_lines_gdf.head(5)

In [ ]:
map_4 = KeplerGl(height=1000)
map_4.add_data(data=POI_matching_lines_gdf, name = "POI matching nodes")
map_4.add_data(data=intersections, name = "Point Of Interests")
map_4.add_data(data=Aimsun_sections, name = "sections")
map_4.add_data(data=Aimsun_nodes, name = "nodes")

map_4

In [ ]:
#Set the location of all the pois of this category. maxdist is the maximum distance that will 
#later be used in find_all_nearest_pois, and maxitems - the maximum number of items that will 
#later be requested in find_all_nearest_pois

#
net.set_pois("All", 300, 20, intersections['Long'], intersections['Lat'])

#Find the distance to the nearest pois from each source node. 
#The bigger values in this case mean less accessibility.
access = net.nearest_pois(300, "All", num_pois=10)

In [ ]:
access.head()

# Accessibility visualization Measures
<a id="accessibility"> </a>

In [ ]:
n = 1
temp_access = access[[n]]
nodes_gdf['cost'] = access[n] 
nodes_gdf.cost[:] = access[n]

## Remark from Theo, I really do not understand why you remove geometry from Aimsun_node to add column x and y, and then add geometry again?
is it because of pandana?

In [ ]:
nodes_gdf['geometry'] = [Point(xy) for xy in zip(nodes_gdf['x'], nodes_gdf['y'])]
nodes_gdf = gpd.GeoDataFrame(nodes_gdf, crs=crs)

In [ ]:
nodes_gdf.head()

#manually create the bins for the dataset only for visualization

In [ ]:
nodes_gdf['cost'].describe()

In [ ]:
map_5 = KeplerGl(height=600)
#map_5.add_data(data=Aimsun_nodes, name = "nodes")
map_5.add_data(data=nodes_gdf[nodes_gdf['cost']>20], name = "accessibility heatmap")
map_5.add_data(data=nodes_gdf[nodes_gdf['cost']<=20], name = "accessibility heatmap_s")
#need manually change to heatmap and let the radius depends on accessibility level
map_5

In [ ]:
net.set_pois("POIs", 300, 20, intersections['Long'], intersections['Lat'])

# access = net.nearest_pois(300, "restaurants", num_pois=10)

# x, y = buildings.x, buildings.y
# buildings["node_ids"] = net.get_node_ids(x, y)
# net.set(node_ids, variable=buildings.square_footage, name="square_footage")
# net.set(node_ids, variable=buildings.residential_units,
#         name="residential_units")

In [ ]:
net.set(intersections['nearest_node_id']) #variable=intersections['count']
aggregated = net.aggregate(60, type='sum', decay='flat') #, imp_name=None, name='tmp'
aggregated

In [ ]:
aggregated.index = range(len(aggregated))

In [ ]:
nodes_gdf.join(aggregated.to_frame())

In [ ]:
aggregated.index = range(len(aggregated))
joined = nodes_gdf.join(aggregated.to_frame())
joined = joined.rename(columns={0:'access_level'})
#joined.plot(column='access_level', cmap='OrRd', figsize=(7, 7))

In [ ]:
joined.head()

In [ ]:
map_6 = KeplerGl(height=600)
map_6.add_data(data=joined[joined['access_level'] < 20], name = "aggregated accessibility heatmap_1")
map_6.add_data(data=joined[joined['access_level']>=20], name = "aggregated accessibility heatmap_2")

map_6

## Creating accessibility isochrone with networkx
<a id="isochrone"> </a>

In [70]:
G = nx.Graph()

In [71]:
edge_weighted.head(5)

,id,id_node_source,id_node_target,capacity,speed,Unnamed: 0,did,oid,ent,exitTime,travelTime,delayTime
0,242,9845,9923,2100,120,5.323860e+05,27989.0,19612.531982,24.076351,59995.530405,26.016289,19.982331
1,243,9852,9848,6300,104,6.488195e+05,27989.0,24440.117725,17.790915,61953.610201,6.407848,0.116653
2,244,9850,9852,6300,104,6.492795e+05,27989.0,24459.738245,16.789826,61953.859779,1.824515,0.079628
3,248,9881,15043,700,50,9.508395e+05,27989.0,37096.313725,3.000000,64670.654902,11.845910,8.478374
4,249,15043,9881,700,50,1.174674e+06,27989.0,44143.833333,29.333333,67056.750000,5.157480,2.083440


## Mark!!!

## To do: use the function from_pandas_dataframe

In [72]:
nodes.head(5)

,id_node,x,y
0,9845,-121.922491,37.495935
1,9848,-121.921727,37.494013
2,9850,-121.923525,37.495613
3,9852,-121.923128,37.495265
4,9854,-121.934705,37.505858


In [75]:
#load the edges and notes to networkx Graph, including the simulated results as attributes
for i in range(len(edge_weighted)):
    id_edge = edge_weighted.iloc[i]['id']
    source_node = edge_weighted.iloc[i]['id_node_source']
    target_node = edge_weighted.iloc[i]['id_node_target']
    distance = edge_weighted.iloc[i]['capacity'] # Here we should get the section distance
    travelTime = edge_weighted.iloc[i]['travelTime']
    G.add_edge(source_node, target_node, id_edge = id_edge, distance=distance, travelTime=travelTime)

In [ ]:
for i in range(len(nodes_gdf)):
    id_node = nodes_gdf.iloc[i]['id_node']
    point = nodes_gdf.iloc[i]['geometry']
#     cost = nodes_gdf.iloc[i]['cost']
    G.add_node(id_node, point = point)
#                , cost = cost)

In [ ]:
def node_getter(index): 
    """
    retrieve the lat and lon of a node at specific index
    """
    return list(G.nodes(data=True))[index][0]
def edge_getter(index):
    """
    retrieve the starting node and endinge node of of an edge at specific index
    """
    u, v, w = list(G.edges(data=True))[index]
    return (u, v, w)


### Applying BFS on generating isochrone

In [ ]:
BFS_trav = list(nx.bfs_edges(G, source=node_getter(1), depth_limit=2))
traversed = []
for t in BFS_trav:
    traversed.append(int(t[0]))
    traversed.append(int(t[1]))
traversed_2 = sorted(list(set(traversed)))

In [ ]:
BFS_trav = list(nx.bfs_edges(G, source=node_getter(1), depth_limit=5))
traversed = []
for t in BFS_trav:
    traversed.append(int(t[0]))
    traversed.append(int(t[1]))
traversed_5 = sorted(list(set(traversed)))

In [ ]:
BFS_trav = list(nx.bfs_edges(G, source=node_getter(1), depth_limit=1))
traversed = []
for t in BFS_trav:
    traversed.append(int(t[0]))
    traversed.append(int(t[1]))
traversed_1 = sorted(list(set(traversed)))

In [ ]:
node_colors = {}
for node in traversed_5:
    node_colors[node] = 3
for node in traversed_2:
    node_colors[node] = 2
for node in traversed_1:
    node_colors[node] = 1
    
nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
#ns = [20 if node in node_colors else 0 for node in G.nodes()]
joined['color'] = nc

In [ ]:
nodes_1 = traversed_1 
nodes_2 = traversed_2
nodes_3 =  traversed_5

In [ ]:
node_getter(1)

## To do: maybe here use alpha shape instead of convex hull

In [ ]:
# Transfering the nodes at different traversal depth to isochrone layer
def traversal_to_layer(node_set):
    layer_nodes_gdf = nodes_gdf[nodes_gdf.id_node.isin(node_set)]
    #generate a convex hull containing all the subgraph nodes 
    bounding_poly = gpd.GeoSeries(layer_nodes_gdf['geometry']).unary_union.convex_hull
    #creating a layer (geopandas df) for this bounding convex hull to be visualized in Kepler.gl
    layer = gpd.GeoDataFrame([bounding_poly], crs=crs)
    layer['geometry'] = layer[0]
    return layer[['geometry']]

In [ ]:
layer_1 = traversal_to_layer(nodes_1)
layer_2 = traversal_to_layer(nodes_2)
layer_3 = traversal_to_layer(nodes_3)

In [ ]:
POIs_gdf

In [ ]:
#calculate the amount of POIs in each layer, applying the gdf.sjoin(): https://geopandas.org/reference/geopandas.sjoin.html

merged = gpd.sjoin(POIs_gdf, layer_1, how='left', op='within')
contained_POIs = merged[merged['index_right'].notna()]
num_contained = len(contained_POIs)

In [ ]:
map_8 = KeplerGl(height=600)
map_8.add_data(data=nodes_gdf[nodes_gdf['id_node'] == 9923], name = "center POI")
map_8.add_data(data=layer_1, name = "isochrone_1")#[joined['color'] == 1]
map_8.add_data(data=layer_2, name = "isochrone_2")
map_8.add_data(data=layer_3, name = "isochrone_3")
map_8.add_data(data=Aimsun_sections, name = "sections")
map_8.add_data(data=Aimsun_nodes, name = "nodes")
map_8

### Using networkx ego graph to plot isochrone: https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.generators.ego.ego_graph.html

In [ ]:
edge_weighted[edge_weighted['id_node_source'] == node_getter(1)]

In [ ]:
subgraph_1 = nx.ego_graph(G, node_getter(1), radius=15, distance='travelTime')
subgraph_2 = nx.ego_graph(G, node_getter(1), radius=20, distance='travelTime')
subgraph_3 = nx.ego_graph(G, node_getter(1), radius=60, distance='travelTime')

In [ ]:
def subgraph_layer(nodes_gdf, subgraph):
    #extract the Nodes dataframe for subgraph
    subgraph_nodes_gdf = nodes_gdf[nodes_gdf.id_node.isin(list(subgraph.nodes))]
    #generate a convex hull containing all the subgraph nodes 
    bounding_poly = gpd.GeoSeries(subgraph_nodes_gdf['geometry']).unary_union.convex_hull
    #creating a layer (geopandas df) for this bounding convex hull to be visualized in Kepler.gl
    layer = gpd.GeoDataFrame([bounding_poly], crs=crs)
    layer['geometry'] = layer[0]
    return layer[['geometry']]

In [ ]:
layer_1 = subgraph_layer(nodes_gdf, subgraph_1)
layer_2 = subgraph_layer(nodes_gdf, subgraph_2)
layer_3 = subgraph_layer(nodes_gdf, subgraph_3)

In [ ]:
map_9 = KeplerGl(height=600)
map_9.add_data(data=nodes_gdf[nodes_gdf['id_node'] == node_getter(1)], name = "all POIs")
# map_8.add_data(data=joined[joined['color'] == 1], name = "isochrone")
# map_8.add_data(data=joined[joined['color'] == 2], name = "isochrone_1")
map_9.add_data(data=Aimsun_sections, name = "sections")
map_9.add_data(data=Aimsun_nodes, name = "nodes")
map_9.add_data(data=layer_1, name = 'layer_1')
map_9.add_data(data=layer_2, name = 'layer_2')
map_9.add_data(data=layer_3, name = 'layer_3')
map_9

For each selected starting POI, we first find and link it to the closest Aimsun network node. 
Then, by running the BFS, we find the range of reachable spots (accessibility area) from this traversal and count the amount of POIs of different categories reachable within the area. 
And develop an algorithm to calculate the accessibility score
